In [4]:
from functools import reduce
import pandas as pd
from polio_utils import download_polio_data, extract_wild_cases, extract_vd_cases, owid_population, standardise_countries


Download and extract the data from the latest wild polio virus pdf

In [2]:
res = download_polio_data(url_stub='https://polioeradication.org/wp-content/uploads/2022/03/weekly-polio-analyses-WPV-')

with open('data/polio_wild_cases.pdf', 'wb') as f:
    f.write(res.content)

wt_df = extract_wild_cases(file_path='data/polio_wild_cases.pdf')

Calculate cases per million population

In [6]:
population = owid_population()
wt_df['entity'] = standardise_countries(wt_df['entity'])
wt_df['year'] = wt_df['year'].astype(int)
wt_df['wild_polio_1_cases'] = wt_df['wild_polio_1_cases'].astype(int)

wt_pop = pd.DataFrame(pd.merge(left = population, right = wt_df, how="right"))
wt_pop['wild_polio_1_cases_per_million'] = (wt_pop['wild_polio_1_cases']/wt_pop['population']) * 1000000
wt_pop['wild_polio_1_cases_per_million'] = wt_pop['wild_polio_1_cases_per_million'].fillna(0).round(decimals=3)

wt_pop

,entity,year,population,wild_polio_1_cases,wild_polio_1_cases_per_million
0,Pakistan,2016,203631360,20,0.098
1,Afghanistan,2016,35383028,13,0.367
2,Malawi,2016,17205254,0,0.000
3,Nigeria,2016,185960256,4,0.022
4,Iran,2016,79563992,0,0.000
5,World,2016,7464344332,37,0.005
6,Pakistan,2017,207906208,8,0.038
7,Afghanistan,2017,36296108,14,0.386
8,Malawi,2017,17670194,0,0.000
9,Nigeria,2017,190873248,0,0.000


Download and extract the data from the latest vaccine derived polio cases pdf

In [8]:
res = download_polio_data(url_stub='http://polioeradication.org/wp-content/uploads/2022/03/weekly-polio-analyses-cVDPV-')

with open('data/polio_vaccine_derived_cases.pdf', 'wb') as f:
    f.write(res.content)

vd_df = extract_vd_cases(file_path = 'data/polio_vaccine_derived_cases.pdf')

In [9]:
population = owid_population()
vd_df['entity'] = standardise_countries(vd_df['entity'])
vd_df['year'] = vd_df['year'].astype(int)

vd_pop = pd.DataFrame(pd.merge(left = population, right = vd_df, how="right"))
vd_pop['cVDPV1_per_million'] = (vd_pop['cVDPV1']/vd_pop['population']) * 1000000
vd_pop['cVDPV2_per_million'] = (vd_pop['cVDPV2']/vd_pop['population']) * 1000000
vd_pop['cVDPV3_per_million'] = (vd_pop['cVDPV3']/vd_pop['population']) * 1000000
vd_pop['total_cVDPV_per_million'] = (vd_pop['total_cVDPV']/vd_pop['population']) * 1000000



In [33]:
polio_dataframes = [wt_pop, vd_pop]

polio_df = reduce(
        lambda left, right: pd.merge(left, right, on=["entity", "year", "population"], how="outer"),
        polio_dataframes,
    )
polio_df = polio_df.drop(columns = ['population'])

In [34]:
cols = polio_df.columns.drop(['entity', 'year'])

int_cols = [k for k in cols if 'per_million' not in k]

float_cols = [k for k in cols if 'per_million' in k]


In [35]:
polio_df[int_cols] = polio_df[int_cols].fillna(0).astype(int)
polio_df[float_cols] = polio_df[float_cols].fillna(0).round(3)

Combining rows where strains are counted separately

In [36]:
polio_df = polio_df.groupby(['entity', 'year']).sum()
polio_df = polio_df.reset_index()


Summing wild and vaccine derived cases

In [44]:
polio_df['total_polio'] = polio_df['wild_polio_1_cases'] + polio_df['total_cVDPV']
polio_df['total_polio_per_million'] = polio_df['wild_polio_1_cases_per_million'] + polio_df['total_cVDPV_per_million']


In [45]:
polio_df

,entity,year,wild_polio_1_cases,wild_polio_1_cases_per_million,cVDPV1,cVDPV2,cVDPV3,total_cVDPV,cVDPV1_per_million,cVDPV2_per_million,cVDPV3_per_million,total_cVDPV_per_million,total_polio,total_polio_per_million
0,Afghanistan,2016,13,0.367,0,0,0,0,0.000,0.000,0.0,0.000,13,0.367
1,Afghanistan,2017,14,0.386,0,0,0,0,0.000,0.000,0.0,0.000,14,0.386
2,Afghanistan,2018,21,0.565,0,0,0,0,0.000,0.000,0.0,0.000,21,0.565
3,Afghanistan,2019,29,0.762,0,0,0,0,0.000,0.000,0.0,0.000,29,0.762
4,Afghanistan,2020,56,1.439,0,308,0,308,0.000,7.912,0.0,7.912,364,9.351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,World,2021,18,0.003,16,622,0,638,0.002,0.079,0.0,0.081,656,0.084
101,Yemen,2019,0,0.000,1,0,0,1,0.034,0.000,0.0,0.034,1,0.034
102,Yemen,2020,0,0.000,31,0,0,31,1.039,0.000,0.0,1.039,31,1.039
103,Yemen,2021,0,0.000,3,13,0,16,0.098,0.426,0.0,0.525,16,0.525


In [ ]:
polio_df.to_csv('data/polio_cases_to_upload.csv', index=False)

Data from 1980 onwards from WHO - download from http://www.who.int/entity/immunization/monitoring_surveillance/data/incidence_series.xls?ua=1

In [47]:
who_polio = pd.read_excel('data/incidence_series.xls', sheet_name='Polio')

In [59]:
who_polio

,WHO_REGION,ISO_code,Cname,Disease,2019,2018,2017,2016,2015,2014,...,1989,1988,1987,1986,1985,1984,1983,1982,1981,1980
0,EMR,AFG,Afghanistan,polio,0.0,0.0,0.0,13.0,20.0,28.0,...,55.0,307.0,628.0,1843.0,1981.0,552.0,1991.0,1390.0,837.0,880.0
1,EUR,ALB,Albania,polio,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
2,AFR,DZA,Algeria,polio,0.0,0.0,0.0,0.0,0.0,0.0,...,18.0,9.0,35.0,29.0,66.0,108.0,132.0,71.0,114.0,116.0
3,EUR,AND,Andorra,polio,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFR,AGO,Angola,polio,138.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,15.0,37.0,14.0,3.0,0.0,6.0,12.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,AMR,VEN,Venezuela (Bolivarian Republic of),polio,0.0,0.0,0.0,0.0,0.0,0.0,...,16.0,17.0,46.0,27.0,8.0,9.0,9.0,30.0,68.0,11.0
190,WPR,VNM,Viet Nam,polio,NaN,NaN,0.0,0.0,0.0,0.0,...,427.0,839.0,1449.0,938.0,1600.0,1158.0,1109.0,897.0,644.0,1741.0
191,EMR,YEM,Yemen,polio,1.0,0.0,0.0,0.0,0.0,0.0,...,701.0,114.0,179.0,601.0,336.0,767.0,633.0,235.0,541.0,722.0
192,AFR,ZMB,Zambia,polio,2.0,0.0,0.0,0.0,0.0,0.0,...,47.0,85.0,69.0,134.0,128.0,177.0,182.0,243.0,429.0,276.0
